In [ ]:
# --- TREINAMENTO CIENTÍFICO DO MODELO DE CLASSIFICAÇÃO MULTICLASSE ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import joblib

print("Iniciando o processo de treinamento do classificador mult classe...")

# --- 1. Carregamento e Preparação dos Dados ---
df = pd.read_csv('dataset-limpo.csv')
df.dropna(subset=['latitude', 'longitude', 'time'], inplace=True)

# Garantir que colunas de itens sejam numéricas (0 ou 1)
item_cols = ['Celular', 'Bolsa ou Mochila', 'Carteira', 'Relógio']
for col in item_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

df['texto_completo'] = df['titulo'].fillna('') + ' ' + df['descricao'].fillna('')
df['hora'] = pd.to_datetime(df['time'], errors='coerce').dt.hour
df.dropna(subset=['hora'], inplace=True)

# --- 2. Engenharia da Variável Alvo (Target) Multiclasse ---
# Aplicando a hierarquia para criar uma única coluna de classe
def definir_classe_crime(row):
    if row['Celular'] == 1:
        return 0  # Celular
    elif row['Bolsa ou Mochila'] == 1:
        return 1  # Bolsa ou Mochila
    elif row['Carteira'] == 1:
        return 2  # Carteira
    elif row['Relógio'] == 1:
        return 3  # Relógio
    else:
        return 4  # Outros

df['classe_crime'] = df.apply(definir_classe_crime, axis=1)

# Mapeamento de classes para nomes
class_names = ['Celular', 'Bolsa ou Mochila', 'Carteira', 'Relógio', 'Outros']
class_map = {i: name for i, name in enumerate(class_names)}

# Analisar a distribuição das novas classes
print("\nDistribuição das 5 classes de crime no dataset completo:")
print(df['classe_crime'].map(class_map).value_counts(normalize=True))

# Definir Features (X) e o novo Alvo (y)
X = df[['latitude', 'longitude', 'hora', 'texto_completo']]
y = df['classe_crime']

# --- 3. Divisão Estratificada em Treino e Teste ---
# A estratificação é ainda mais crucial aqui para preservar a proporção de todas as 5 classes.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
print(f"\nDados divididos: {len(X_train)} para treino, {len(X_test)} para teste.")

# --- 4. Construção do Pipeline de Pré-processamento e Modelagem ---
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', StandardScaler(), ['latitude', 'longitude', 'hora']),
        ('text', TfidfVectorizer(max_features=2000, ngram_range=(1,2)), 'texto_completo')
    ],
    remainder='drop'
)

# O class_weight='balanced' ajuda o modelo a dar a devida importância às classes menores.
pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1))
])

# --- 5. Otimização de Hiperparâmetros com GridSearchCV ---
# Usaremos 'f1_weighted' como métrica, que é excelente para problemas mult classe desbalanceados.
param_grid = {
    'classifier__n_estimators': [150, 250],
    'classifier__max_depth': [20, 30],
    'preprocessor__text__max_features': [2000, 3000]
}

print("\nIniciando a busca de hiperparâmetros com GridSearchCV (pode demorar)...")
grid_search = GridSearchCV(pipeline_rf, param_grid, cv=3, n_jobs=-1, verbose=2, scoring='f1_weighted')
grid_search.fit(X_train, y_train)

# --- 6. Treinamento Final e Avaliação no Conjunto de Teste ---
print("\nMelhores parâmetros encontrados pela validação cruzada:")
print(grid_search.best_params_)

best_model = grid_search.best_estimator_

print("\nAvaliando o melhor modelo no conjunto de teste (dados nunca vistos antes)...")
y_pred = best_model.predict(X_test)

# Exibe o relatório de classificação final para todas as 5 classes
print("\nRelatório de Classificação Final (Mult Classe):")
print(classification_report(y_test, y_pred, target_names=class_names))

# --- 7. Visualização e Salvamento ---
print("Gerando Matriz de Confusão (5x5)...")
fig, ax = plt.subplots(figsize=(12, 10))
ConfusionMatrixDisplay.from_estimator(
    best_model,
    X_test,
    y_test,
    ax=ax,
    cmap='Blues',
    display_labels=class_names,
    xticks_rotation='vertical'
)
plt.title("Matriz de Confusão no Conjunto de Teste")
plt.tight_layout()
plt.show()

# Salva o pipeline final, que agora é um modelo multiclasse
joblib.dump(best_model, 'modelo_classificacao_multiclasse_rf.joblib')
print("\nModelo de classificação multiclasse final salvo como 'modelo_classificacao_multiclasse_rf.joblib'")

Iniciando o processo de treinamento do classificador...
Dados divididos: 6282 para treino, 2095 para teste.
Iniciando a busca de hiperparâmetros com GridSearchCV (pode demorar)...
Fitting 3 folds for each of 8 candidates, totalling 24 fits


c:\Users\ruam_\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan]
  warnings.warn(



Melhores parâmetros encontrados pela validação cruzada:
{'classifier__max_depth': 20, 'classifier__n_estimators': 100, 'preprocessor__text__max_features': 1500}

Avaliando o melhor modelo no conjunto de teste (dados nunca vistos antes)...

Relatório de Classificação Final:


ValueError: Number of classes, 1, does not match size of target_names, 2. Try specifying the labels parameter